## --- Day 22: Monkey Map ---

The monkeys take you on a surprisingly easy trail through the jungle. They're even going in roughly the right direction according to your handheld device's Grove Positioning System.

As you walk, the monkeys explain that the grove is protected by a force field. To pass through the force field, you have to enter a password; doing so involves tracing a specific path on a strangely-shaped board.

At least, you're pretty sure that's what you have to do; the elephants aren't exactly fluent in monkey.

The monkeys give you notes that they took when they last saw the password entered (your puzzle input).

For example:
```
        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
```
The first half of the monkeys' notes is a map of the board. It is comprised of a set of open tiles (on which you can move, drawn .) and solid walls (tiles which you cannot enter, drawn #).

The second half is a description of the path you must follow. It consists of alternating numbers and letters:

 -   A number indicates the number of tiles to move in the direction you are facing. If you run into a wall, you stop moving forward and continue with the next instruction.
 -   A letter indicates whether to turn 90 degrees clockwise (R) or counterclockwise (L). Turning happens in-place; it does not change your current tile.

So, a path like 10R5 means "go forward 10 tiles, then turn clockwise 90 degrees, then go forward 5 tiles".

You begin the path in the leftmost open tile of the top row of tiles. Initially, you are facing to the right (from the perspective of how the map is drawn).

If a movement instruction would take you off of the map, you wrap around to the other side of the board. In other words, if your next tile is off of the board, you should instead look in the direction opposite of your current facing as far as you can until you find the opposite edge of the board, then reappear there.

For example, if you are at A and facing to the right, the tile in front of you is marked B; if you are at C and facing down, the tile in front of you is marked D:
```
        ...#
        .#..
        #...
        ....
...#.D.....#
........#...
B.#....#...A
.....C....#.
        ...#....
        .....#..
        .#......
        ......#.
```
It is possible for the next tile (after wrapping around) to be a wall; this still counts as there being a wall in front of you, and so movement stops before you actually wrap to the other side of the board.

By drawing the last facing you had with an arrow on each tile you visit, the full path taken by the above example looks like this:
```
        >>v#    
        .#v.    
        #.v.    
        ..v.    
...#...v..v#    
>>>v...>#.>>    
..#v...#....    
...>>>>v..#.    
        ...#....
        .....#..
        .#......
        ......#.
```
To finish providing the password to this strange input device, you need to determine numbers for your final row, column, and facing as your final position appears from the perspective of the original map. Rows start from 1 at the top and count downward; columns start from 1 at the left and count rightward. (In the above example, row 1, column 1 refers to the empty space with no tile on it in the top-left corner.) Facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^). The final password is the sum of 1000 times the row, 4 times the column, and the facing.

In the above example, the final row is 6, the final column is 8, and the final facing is 0. So, the final password is 1000 * 6 + 4 * 8 + 0: 6032.

Follow the path given in the monkeys' notes. What is the final password?


In [169]:
import re

# Directions are integers
# 0 Right 1 Down 2 Left 3 Up

f = open("input.txt", "r")
lines = f.read().splitlines()

my_map = []
path_next = False
instructions = []
for line in lines:
    if line == "":
        path_next = True
    elif path_next:
        instructions = re.findall('\d+|L|R', line)
    else:
        my_map.append(line)

position = (0, my_map[0].index("."))
direction = 0

def change_direction(direction, way):
    direction = (direction + 1)%4 if way == "R" else (direction - 1)%4
    return direction

def move(): # Return boolean if could move or not
    if direction == 0:
        return move_right()
    elif direction == 1:
        return move_down()
    elif direction == 2:
        return move_left()
    elif direction == 3:
        return move_up()
    
def get_min_y_index(char):
    for row_idx in range(len(my_map)):
        if position[1] >= len(my_map[row_idx]):
            continue
        else:
            token = my_map[row_idx][position[1]]
            if token == char:
                return row_idx
    print("Error getting min_y_index for", char, "at", position)
    return -1

def get_max_y_index(char):
    for row_idx in range(len(my_map) - 1, -1, -1):
        if position[1] >= len(my_map[row_idx]):
            continue
        else:
            token = my_map[row_idx][position[1]]
            if token == char:
                return row_idx
    print("Error getting max_y_index for", char, "at", position)
    return -1

def get_min_x_index(char):
    return my_map[position[0]].index(char)

def get_max_x_index(char):
    return my_map[position[0]].rindex(char)

In [78]:
def move_right():
    global position
    row = my_map[position[0]]

    if position[1] + 1 == len(row): # Reached end of line
        wall_index = get_min_x_index("#")
        floor_index = get_min_x_index(".")

        if floor_index < wall_index:
            position = (position[0], floor_index)
            return True
        else:
            return False
    else: # Normal walking
        char = row[position[1] + 1]
        if   char == "#":
            return False
        elif char  == ".":
            position = (position[0], position[1] + 1)
            return True            

def move_left():
    global position
    row = my_map[position[0]]

    if position[1] == 0 or row[position[1] - 1] == " ": # Reached end of line
        wall_index = get_max_x_index("#")
        floor_index = get_max_x_index(".")

        if floor_index > wall_index:
            position = (position[0], floor_index)
            return True
        else:
            return False
    else: # Normal walking
        char = row[position[1] - 1]
        if   char == "#":
            return False
        elif char  == ".":
            position = (position[0], position[1] - 1)
            return True
        
def move_down():
    global position
    if position[0] + 1 == len(my_map) or position[1] >= len(my_map[position[0] + 1]) or my_map[position[0] + 1][position[1]] == " ":
        wall_index = get_min_y_index("#")
        floor_index = get_min_y_index(".")
        
        if floor_index < wall_index:
            position = (floor_index, position[1])
            return True
        else:
            return False
    else: # Normal walking
        char = my_map[position[0] + 1][position[1]]
        if char == ".":
            position = (position[0] + 1, position[1])
            return True
        else:
            return False

def move_up():
    global position
    if position[0] == 0 or position[1] >= len(my_map[position[0] - 1]) or my_map[position[0] - 1][position[1]] == " ":
        wall_index = get_max_y_index("#")
        floor_index = get_max_y_index(".")
        
        if floor_index > wall_index:
            position = (floor_index, position[1])
            return True
        else:
            return False
    else: # Normal walking
        char = my_map[position[0] - 1][position[1]]
        if char == ".":
            position = (position[0] - 1, position[1])
            return True
        else:
            return False

In [79]:
for instruction in instructions:
    if instruction == "L" or instruction == "R":
        direction = change_direction(direction, instruction)
    else:
        moves = int(instruction)
        while(moves):
            if not move():
                break
            moves -= 1
1000*(position[0]+1) + 4*(position[1]+1) + direction

6032

## --- Part Two ---

As you reach the force field, you think you hear some Elves in the distance. Perhaps they've already arrived?

You approach the strange input device, but it isn't quite what the monkeys drew in their notes. Instead, you are met with a large cube; each of its six faces is a square of 50x50 tiles.

To be fair, the monkeys' map does have six 50x50 regions on it. If you were to carefully fold the map, you should be able to shape it into a cube!

In the example above, the six (smaller, 4x4) faces of the cube are:
```
        1111
        1111
        1111
        1111
222233334444
222233334444
222233334444
222233334444
        55556666
        55556666
        55556666
        55556666
```
You still start in the same position and with the same facing as before, but the wrapping rules are different. Now, if you would walk off the board, you instead proceed around the cube. From the perspective of the map, this can look a little strange. In the above example, if you are at A and move to the right, you would arrive at B facing down; if you are at C and move down, you would arrive at D facing up:
```
        ...#
        .#..
        #...
        ....
...#.......#
........#..A
..#....#....
.D........#.
        ...#..B.
        .....#..
        .#......
        ..C...#.
```
Walls still block your path, even if they are on a different face of the cube. If you are at E facing up, your movement is blocked by the wall marked by the arrow:
```
        ...#
        .#..
     -->#...
        ....
...#..E....#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.
```
Using the same method of drawing the last facing you had with an arrow on each tile you visit, the full path taken by the above example now looks like this:
```
        >>v#    
        .#v.    
        #.v.    
        ..v.    
...#..^...v#    
.>>>>>^.#.>>    
.^#....#....    
.^........#.    
        ...#..v.
        .....#v.
        .#v<<<<.
        ..v...#.
```
The final password is still calculated from your final position and facing from the perspective of the map. In this example, the final row is 5, the final column is 7, and the final facing is 3, so the final password is 1000 * 5 + 4 * 7 + 3 = 5031.

Fold the map into a cube, then follow the path given in the monkeys' notes. What is the final password?

In [170]:
def move_in_cube(): # Return boolean if could move or not
    global cube_pos
    global direction
    
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    
    new_pos = (cube_pos[1] + directions[direction][0], cube_pos[2] + directions[direction][1])
    #print(cube_pos, direction, new_pos)
    if new_pos[0] == -1 or new_pos[0] == face_length or new_pos[1] == -1 or new_pos[1] == face_length:
        wrap_info = wraps[cube_pos[0]][direction]
        
        x = cube_pos[2]
        y = cube_pos[1]
        
        if wrap_info["flip_x"]:
            x = face_length - x - 1
        if wrap_info["flip_y"]:
            y = face_length - y - 1
        if wrap_info["swap_xy"]:
            temp = x
            x = y
            y = temp
        
        char = faces[wrap_info["face"]][y][x]
        
        if char == ".":
            cube_pos = (wrap_info["face"], y, x)
            direction = wrap_info["dir"]
            return True
        else:
            return False
    else:
        char = faces[cube_pos[0]][ cube_pos[1] + directions[direction][0] ][ cube_pos[2] + directions[direction][1] ]
        if char == ".":
            cube_pos = (cube_pos[0], cube_pos[1] + directions[direction][0], cube_pos[2] + directions[direction][1])
            return True
        else:
            return False

In [167]:
## Part 2 preparation FOR TEST INPUT
face_length = len(my_map[0])//3
# 0 is Up, 1 is Back, 2 is Left, 3 is Front, 4 is Down, 5 is Right
faces = [[], [], [], [], [], []]

for i in range(3*face_length):
    if i < face_length:
        faces[0].append(my_map[i][2*face_length:])
    elif i >= 2*face_length:
        faces[4].append(my_map[i][2*face_length:3*face_length])
        faces[5].append(my_map[i][3*face_length:])
    else:
        faces[1].append(my_map[i][:face_length])
        faces[2].append(my_map[i][face_length:2*face_length])
        faces[3].append(my_map[i][2*face_length:])
    print(my_map[i])

wraps = {
    0: {
        0: {"face": 5, "dir": 2, "flip_x": False, "flip_y": True, "swap_xy": False},
        1: {"face": 3, "dir": 1, "flip_x": False, "flip_y": True, "swap_xy": False},
        2: {"face": 2, "dir": 1, "flip_x": False, "flip_y": False, "swap_xy": True},
        3: {"face": 1, "dir": 1, "flip_x": True, "flip_y": False, "swap_xy": False},
    },
    1: {
        0: {"face": 2, "dir": 0, "flip_x": True, "flip_y": False, "swap_xy": False},
        1: {"face": 4, "dir": 3, "flip_x": True, "flip_y": False, "swap_xy": False},
        2: {"face": 5, "dir": 3, "flip_x": True, "flip_y": True, "swap_xy": True},
        3: {"face": 0, "dir": 1, "flip_x": True, "flip_y": False, "swap_xy": False},
    },
    2: {
        0: {"face": 3, "dir": 0, "flip_x": True, "flip_y": False, "swap_xy": False},
        1: {"face": 4, "dir": 0, "flip_x": True, "flip_y": True, "swap_xy": True},
        2: {"face": 1, "dir": 2, "flip_x": True, "flip_y": False, "swap_xy": False},
        3: {"face": 0, "dir": 0, "flip_x": False, "flip_y": False, "swap_xy": True},
    },
    3: {
        0: {"face": 5, "dir": 1, "flip_x": True, "flip_y": True, "swap_xy": True},
        1: {"face": 4, "dir": 1, "flip_x": False, "flip_y": True, "swap_xy": False},
        2: {"face": 2, "dir": 2, "flip_x": True, "flip_y": False, "swap_xy": False},
        3: {"face": 0, "dir": 3, "flip_x": False, "flip_y": True, "swap_xy": False},
    },
    4: {
        0: {"face": 5, "dir": 0, "flip_x": True, "flip_y": False, "swap_xy": False},
        1: {"face": 1, "dir": 3, "flip_x": True, "flip_y": False, "swap_xy": False},
        2: {"face": 2, "dir": 3, "flip_x": True, "flip_y": True, "swap_xy": True},
        3: {"face": 3, "dir": 3, "flip_x": False, "flip_y": True, "swap_xy": False},
    },
    5: {
        0: {"face": 0, "dir": 2, "flip_x": False, "flip_y": True, "swap_xy": False},
        1: {"face": 1, "dir": 0, "flip_x": True, "flip_y": True, "swap_xy": True},
        2: {"face": 4, "dir": 2, "flip_x": True, "flip_y": False, "swap_xy": False},
        3: {"face": 3, "dir": 2, "flip_x": True, "flip_y": True, "swap_xy": True},
    },
}

cube_pos = (0, 0, 0)
direction = 0

for instruction in instructions:
    if instruction == "L" or instruction == "R":
        direction = change_direction(direction, instruction)
    else:
        moves = int(instruction)
        while(moves):
            if not move_in_cube():
                break
            moves -= 1

print(direction)
print(cube_pos)

final_row = 1*face_length + cube_pos[1] + 1
final_col = 1*face_length + cube_pos[2] + 1
print(final_row, final_col)
1000*final_row + 4*final_col + direction

        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.
3
(2, 0, 2)
5 7


5031

In [159]:
final_col

13

In [173]:
## Part 2 preparation FOR REAL INPUT
face_length = len(my_map[0])//3
# 0 is Up, 1 is Back, 2 is Left, 3 is Front, 4 is Down, 5 is Right
faces = [[], [], [], [], [], []]

for i in range(4*face_length):
    if i < face_length:
        faces[0].append(my_map[i][face_length:2*face_length])
        faces[1].append(my_map[i][2*face_length:])
    elif i >= face_length and i < 2*face_length:
        faces[2].append(my_map[i][face_length:2*face_length])
    elif i >= 2*face_length and i < 3*face_length:
        faces[3].append(my_map[i][:face_length])
        faces[4].append(my_map[i][face_length:2*face_length])
    else:
        faces[5].append(my_map[i][:face_length])        

wraps = {
    0: {
        0: {"face": 1, "dir": 0, "flip_x": True, "flip_y": True, "swap_xy": False},
        1: {"face": 2, "dir": 1, "flip_x": False, "flip_y": True, "swap_xy": False},
        2: {"face": 3, "dir": 0, "flip_x": False, "flip_y": True, "swap_xy": False},
        3: {"face": 5, "dir": 0, "flip_x": False, "flip_y": False, "swap_xy": True},
    },
    1: {
        0: {"face": 4, "dir": 2, "flip_x": False, "flip_y": True, "swap_xy": False},
        1: {"face": 2, "dir": 2, "flip_x": False, "flip_y": False, "swap_xy": True},
        2: {"face": 0, "dir": 2, "flip_x": True, "flip_y": False, "swap_xy": False},
        3: {"face": 5, "dir": 3, "flip_x": False, "flip_y": True, "swap_xy": False},
    },
    2: {
        0: {"face": 1, "dir": 3, "flip_x": False, "flip_y": False, "swap_xy": True},
        1: {"face": 4, "dir": 1, "flip_x": False, "flip_y": True, "swap_xy": False},
        2: {"face": 3, "dir": 1, "flip_x": False, "flip_y": False, "swap_xy": True},
        3: {"face": 0, "dir": 3, "flip_x": False, "flip_y": True, "swap_xy": False},
    },
    3: {
        0: {"face": 4, "dir": 0, "flip_x": True, "flip_y": False, "swap_xy": False},
        1: {"face": 5, "dir": 1, "flip_x": False, "flip_y": True, "swap_xy": False},
        2: {"face": 0, "dir": 0, "flip_x": False, "flip_y": True, "swap_xy": False},
        3: {"face": 2, "dir": 0, "flip_x": False, "flip_y": False, "swap_xy": True},
    },
    4: {
        0: {"face": 1, "dir": 2, "flip_x": False, "flip_y": True, "swap_xy": False},
        1: {"face": 5, "dir": 2, "flip_x": False, "flip_y": False, "swap_xy": True},
        2: {"face": 3, "dir": 2, "flip_x": True, "flip_y": False, "swap_xy": False},
        3: {"face": 2, "dir": 3, "flip_x": False, "flip_y": True, "swap_xy": False},
    },
    5: {
        0: {"face": 4, "dir": 3, "flip_x": False, "flip_y": False, "swap_xy": True},
        1: {"face": 1, "dir": 1, "flip_x": False, "flip_y": True, "swap_xy": False},
        2: {"face": 0, "dir": 1, "flip_x": False, "flip_y": False, "swap_xy": True},
        3: {"face": 3, "dir": 3, "flip_x": False, "flip_y": True, "swap_xy": False},
    },
}

cube_pos = (0, 0, 0)
direction = 0

for instruction in instructions:
    if instruction == "L" or instruction == "R":
        direction = change_direction(direction, instruction)
    else:
        moves = int(instruction)
        while(moves):
            if not move_in_cube():
                break
            moves -= 1

print(direction)
cube_pos

0


(4, 41, 6)

In [175]:
final_row = 2*face_length + cube_pos[1] + 1
final_col = 1*face_length + cube_pos[2] + 1

1000*final_row + 4*final_col + direction

142228

In [168]:
187186

187186